# Description

TODO

# Modules

In [1]:
import pandas as pd

import conf

# Settings

# Paths

In [2]:
INPUT_GWAS_DIR = conf.GWAS_DIR / "asthma-copd"
display(INPUT_GWAS_DIR)
assert INPUT_GWAS_DIR.exists()

PosixPath('/opt/data/data/gwas/asthma-copd')

# List GWAS dir

In [3]:
list(INPUT_GWAS_DIR.iterdir())

[PosixPath('/opt/data/data/gwas/asthma-copd/GWAS_ACO_GLM_SNPs_info0.7.txt'),
 PosixPath('/opt/data/data/gwas/asthma-copd/GWAS_Asthma_only_GLM_SNPs_info0.7.txt'),
 PosixPath('/opt/data/data/gwas/asthma-copd/GWAS_COPD_only_GLM_SNPs_info0.7.txt'),
 PosixPath('/opt/data/data/gwas/asthma-copd/README.txt'),
 PosixPath('/opt/data/data/gwas/asthma-copd/GWAS_subjects_summary.pdf')]

# Read ACO GWAS

These GWAS results were generated using PLINK 2.0

In [4]:
gwas_data = pd.read_csv(INPUT_GWAS_DIR / "GWAS_ACO_GLM_SNPs_info0.7.txt", sep="\t")

In [5]:
gwas_data.shape

(8512179, 15)

In [6]:
gwas_data.head()

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P
0,10,60684,rs569167217,A,C,C,N,ADD,169680,1.038160,0.062967,0.917626,1.17452,0.594731,0.552023
1,10,61331,rs548639866,A,G,G,N,ADD,169680,1.037200,0.062959,0.916791,1.17341,0.580072,0.561866
2,10,63213,rs542543788,G,C,C,N,ADD,169680,1.038160,0.062967,0.917626,1.17452,0.594731,0.552023
3,10,64869,rs556434813,C,A,A,N,ADD,169680,1.053000,0.083741,0.893607,1.24082,0.616667,0.537454
4,10,66326,rs12260013,A,G,G,N,ADD,169680,0.940703,0.059882,0.836529,1.05785,-1.020800,0.307349


See here for PLINK formats: https://www.cog-genomics.org/plink/2.0/formats

Some notes on columns:
* `A1` is the counted allele (effect allele in PrediXcan terms).
* `OBS_CT` is the sample size
* `L95` and `U95` is the confidence interval for `OR`.

## Checks

### Stats

In [20]:
_tmp_p_stats = gwas_data["P"].describe()
display(_tmp_p_stats)

assert _tmp_p_stats["min"] > 0.00
assert _tmp_p_stats["max"] <= 1.00

count    8.512179e+06
mean     4.797262e-01
std      2.950372e-01
min      2.264950e-33
25%      2.192630e-01
50%      4.733590e-01
75%      7.354680e-01
max      1.000000e+00
Name: P, dtype: float64

In [21]:
gwas_data["Z_STAT"].describe()

count    8.512179e+06
mean     6.661494e-03
std      1.108981e+00
min     -1.203720e+01
25%     -7.137015e-01
50%      1.694920e-03
75%      7.203110e-01
max      1.198630e+01
Name: Z_STAT, dtype: float64

### Sample size

In [24]:
gwas_data["OBS_CT"].unique()

array([169680])

Sample size column has a single value.

### Number of chromosomes

In [7]:
assert gwas_data["#CHROM"].unique().shape[0] == 22

### Repeated chr/pos rows

In [11]:
duplicated_rows = gwas_data[gwas_data.duplicated(subset=["#CHROM", "POS"], keep=False)]
display(duplicated_rows)

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P
1063,10,504878,rs10795167,G,A,G,N,ADD,169680,0.997741,0.046557,0.910727,1.093070,-0.048576,0.961257
1064,10,504878,rs10795167,G,C,G,N,ADD,169680,0.981176,0.023126,0.937695,1.026670,-0.821734,0.411228
1606,10,679195,rs816568,T,A,A,N,ADD,169680,1.009850,0.034597,0.943639,1.080700,0.283192,0.777030
1607,10,679195,rs816568,T,C,T,N,ADD,169680,1.047530,0.042594,0.963633,1.138740,1.090260,0.275599
2144,10,796886,rs3118490,T,A,A,N,ADD,169680,0.994580,0.053801,0.895043,1.105190,-0.101023,0.919532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8508770,9,139817331,rs4880165,A,T,T,N,ADD,169680,0.900612,0.079223,0.771087,1.051890,-1.321350,0.186384
8509971,9,140247497,9:140247497_A_C,A,C,C,N,ADD,169680,0.954382,0.026373,0.906304,1.005010,-1.770470,0.076649
8509972,9,140247497,9:140247497_A_T,A,T,T,N,ADD,169680,0.922489,0.039547,0.853687,0.996836,-2.040100,0.041340
8511713,9,140911199,9:140911199_C_A,C,A,A,N,ADD,169680,1.024170,0.021989,0.980970,1.069280,1.086190,0.277395


In [12]:
duplicated_rows["P"].describe()

count    1.114200e+04
mean     4.789023e-01
std      2.940300e-01
min      1.237170e-29
25%      2.237248e-01
50%      4.711800e-01
75%      7.337515e-01
max      9.998780e-01
Name: P, dtype: float64

In [22]:
duplicated_rows.sort_values(by="P").head(10)

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P
6734421,6,32612430,rs6926894,T,C,T,N,ADD,169680,0.816325,0.017951,0.788103,0.845558,-11.30520,1.237170e-29
6735942,6,32635903,rs3828794,G,C,G,N,ADD,169680,0.818518,0.018149,0.789914,0.848159,-11.03410,2.617490e-28
6733810,6,32606885,rs9272545,G,T,G,N,ADD,169680,0.818146,0.018280,0.789352,0.847989,-10.98020,4.760060e-28
6734544,6,32613824,rs9273226,C,T,C,N,ADD,169680,0.835807,0.017889,0.807010,0.865631,-10.02640,1.167190e-23
6734158,6,32610009,rs2308891,C,A,A,N,ADD,169680,1.191540,0.017967,1.150310,1.234250,9.75380,1.776930e-22
6735941,6,32635903,rs3828794,G,A,A,N,ADD,169680,1.197240,0.019026,1.153420,1.242730,9.46165,3.031270e-21
6734159,6,32610009,rs2308891,C,G,G,N,ADD,169680,1.182430,0.018817,1.139620,1.226860,8.90551,5.314340e-19
6734392,6,32612114,rs9273060,A,G,G,N,ADD,169680,1.164150,0.017987,1.123820,1.205920,8.44983,2.917250e-17
6735540,6,32631612,rs9274267,A,T,T,N,ADD,169680,1.167050,0.019619,1.123030,1.212800,7.87391,3.437190e-15
6730038,6,32550203,rs34077178,T,G,T,N,ADD,169680,0.849591,0.021039,0.815269,0.885357,-7.74748,9.373720e-15


In cases like `rs3828794` (chr 6), REF=G, ALT=C, A1=G, should not be taken as REF/A1, since it would be G/G.

Here we need to change those rows and make sure we have the correct alleles.

### Check multiallelic variants

In [28]:
_tmp = gwas_data[gwas_data["REF"].str.contains(",")]
display(_tmp)

assert _tmp.shape[0] == 0

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P


In [26]:
_tmp = gwas_data[gwas_data["ALT"].str.contains(",")]
display(_tmp)

assert _tmp.shape[0] == 0

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P


In [27]:
_tmp = gwas_data[gwas_data["A1"].str.contains(",")]
display(_tmp)

assert _tmp.shape[0] == 0

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P


### Check A1 (effect allele)

In [25]:
gwas_data[gwas_data["ALT"] != gwas_data["A1"]]

,#CHROM,POS,ID,REF,ALT,A1,FIRTH?,TEST,OBS_CT,OR,LOG(OR)_SE,L95,U95,Z_STAT,P
6,10,67774,rs9943471,G,C,G,N,ADD,169680,1.026860,0.022695,0.982189,1.07357,1.168070,0.242779
7,10,68533,rs35819232,T,G,T,N,ADD,169680,1.014360,0.022561,0.970482,1.06022,0.631928,0.527434
8,10,69083,rs35418599,C,T,C,N,ADD,169680,1.026960,0.021895,0.983818,1.07198,1.214860,0.224421
9,10,69486,rs34485355,G,A,G,N,ADD,169680,1.035800,0.021679,0.992710,1.08076,1.622440,0.104708
10,10,71350,rs36120673,A,G,A,N,ADD,169680,1.029630,0.022397,0.985415,1.07584,1.303940,0.192253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8512173,9,141105225,9:141105225_G_A,G,A,G,N,ADD,169680,1.004030,0.020829,0.963864,1.04586,0.192948,0.847000
8512174,9,141107003,9:141107003_C_T,C,T,C,N,ADD,169680,0.985896,0.020009,0.947980,1.02533,-0.709872,0.477783
8512175,9,141107896,9:141107896_C_T,C,T,C,N,ADD,169680,0.983620,0.019627,0.946500,1.02220,-0.841451,0.400095
8512176,9,141108111,9:141108111_T_C,T,C,T,N,ADD,169680,0.979441,0.019798,0.942164,1.01819,-1.049290,0.294046


Looks like A1 is not always equal to ALT, so I need to adjust that

# Harmonization, etc

In this page (https://predictdb.org/post/2021/07/21/gtex-v8-models-on-eqtl-and-sqtl/) they recommend two links:
* https://github.com/hakyimlab/MetaXcan/wiki/Best-practices-for-integrating-GWAS-and-GTEX-v8-transcriptome-prediction-models
* Tutorial: https://github.com/hakyimlab/MetaXcan/wiki/Tutorial:-GTEx-v8-MASH-models-integration-with-a-Coronary-Artery-Disease-GWAS

# Save GWAS in common format

BELOW SHOULD GO TO ANOTHER NOTEBOOK

# Run S-PrediXcan

Take a look here: https://github.com/hakyimlab/phenomexcan/blob/master/scripts/000_spredixcan/jobs/02_phenotypes_processing/main

```
        python ${METAXCAN_DIR}/software/MetaXcan.py \
            --model_db_path ${METAXCAN_GTEX_V8_MODELS_DIR}/${METAXCAN_GTEX_V8_FILES_PREFIX}${tissue}.db \
            --covariance ${METAXCAN_GTEX_V8_COVARS_DIR}/${METAXCAN_GTEX_V8_FILES_PREFIX}${tissue}.txt.gz \
            --gwas_folder ${WORK_DIR} \
            --gwas_file_pattern "final.*tsv" \
            --separator $'\t' \
            --non_effect_allele_column "ref" \
            --effect_allele_column "alt" \
            --snp_column  "${METAXCAN_RUN_SNP_COLUMN}" \
            --beta_column "beta" \
            --se_column "se" \
            ${METAXCAN_RUN_EXTRA_OPTIONS} --output_file ${RESULT_PHENO_DIR}/${CSV_FILENAME} >> ${RESULT_PHENO_DIR}/${LOG_FILENAME} 2>&1

```